In [13]:
# import libraries and set max column width as 100 to display 
#import plotly.offline as py
import plotly.plotly as py
#py.init_notebook_mode(connected=True)
import plotly
import getpass 
import pandas as pd
import os
import re

plotly.tools.set_credentials_file(username='jonamjar', api_key='XZzdsQrL8FSGjv7b8Hu3')
pd.set_option("display.max_colwidth", 40)
pd.set_option("display.max_columns", 100)
pd.set_option("display.max_rows", 100)

In [14]:
CSS = """
div.cell:nth-child(5) .output {
    flex-direction: row;}
"""

In [15]:
# get data

ntf_training_df = pd.read_csv("resources/ntf_training_data.csv", low_memory=0)
customer_training_data = pd.read_csv("resources/customer_training_data.csv", low_memory=0)
states = pd.read_csv("resources/states.csv")

print(" \nNTF Training Data")
display(ntf_training_df.head())
print(" \nCustomer Training Data")
display(customer_training_data.head())

 
NTF Training Data


,customer_id,zip_code,city,state,region,estimated_population,census_income,first_order_date,first_order_channel,first_order_md,first_order_brand,first_order_sales,num_orders_2years,total_sales_2years
0,1724258,84109,Salt Lake City,UT,Rockies,19819.0,48122.08,2015-06-25 21:37:07,CS,Men's,The North Face,0.00,8,895.25
1,105640784,72764,Springdale,AR,Midwest (South),33654.0,31827.30,2015-01-04 15:43:07,PLA,Ski,Giro,149.90,1,149.90
2,105669196,77301,Conroe,TX,South West,18057.0,25785.79,2015-01-11 13:19:22,PLA,Accessories,Sol Republic,77.97,1,77.97
3,105671761,02764,North Dighton,MA,North East,3638.0,45405.48,2015-01-11 22:19:22,Organic,Clothing Accessories,Oakley,24.00,1,24.00
4,105681094,57783,Spearfish,SD,Midwest (North),11707.0,31115.96,2015-01-14 10:19:22,Organic,Women's,The North Face,159.95,3,280.68


 
Customer Training Data


,customer_id,zip_code,city,state,region,estimated_population,census_income,first_order_date,first_order_channel,first_order_md,first_order_brand,last_order_date,total_sales,num_orders,num_highcost_orders,num_lowcost_orders,num_cs_orders,num_ghxconnected_orders,num_orders_2017,total_sales_2017
0,106553549,02448,Unknown,NaN,Unknown,0.0,NaN,2015-09-28 19:19:19,Direct,Fly Fishing,Patagonia,2015-09-28 19:19:19,114.56,1,0,1,0,0,0,0.0
1,106892344,78528,Unknown,NaN,Unknown,0.0,NaN,2015-11-30 18:55:04,CS,Women's,The North Face,2015-11-30 18:55:04,75.01,1,0,0,1,0,0,0.0
2,106823206,30862,Unknown,NaN,Unknown,0.0,NaN,2015-11-27 14:01:17,PLA,Men's,Patagonia,2015-11-27 14:01:17,48.30,1,1,0,0,0,0,0.0
3,105813889,01766,Unknown,NaN,Unknown,0.0,NaN,2015-11-27 21:00:19,Direct,Women's,Marmot,2016-12-17 21:42:04,819.52,3,0,2,1,0,0,0.0
4,106781148,09160,Unknown,NaN,Unknown,0.0,NaN,2015-11-23 15:44:34,Direct,Women's,The North Face,2015-11-23 15:44:34,71.96,1,0,1,0,0,0,0.0


In [16]:
# grouping data by state 

state_total_sales_2years = ntf_training_df[["state", "total_sales_2years"]].groupby("state").mean().reset_index()

In [17]:
# data after grouping

display(state_total_sales_2years.head())

# total number of states in the data set
print(" Total number of states in the datased: %d"%len(list(set(state_total_sales_2years["state"]))))
#print(state_total_sales_2years['state'])

,state,total_sales_2years
0,AA,135.804000
1,AE,416.839811
2,AK,332.134947
3,AL,190.981618
4,AP,455.611692


 Total number of states in the datased: 61


In [18]:
# Total states

states = states.sort_values(by = 'State',ascending=True)
states.columns = ['state_name', 'state']

In [19]:
# States discripency

states_joined = pd.merge(state_total_sales_2years, states, on = 'state', how = 'left')

print('What are these state codes?\n')
print(states_joined[~(states_joined['state_name'].notnull())]["state"])

What are these state codes?

0     AA
1     AE
4     AP
6     AS
14    FM
16    GU
31    MP
46    PR
47    PW
55    VI
Name: state, dtype: object


In [20]:
# removing weird state codes

state_total_sales_2years = pd.merge(state_total_sales_2years, states, on = 'state', how = 'inner')

In [21]:
for col in state_total_sales_2years.columns:
    state_total_sales_2years[col] = state_total_sales_2years[col].astype(str)

scl = [[0.0, 'rgb(242,240,247)'],[0.2, 'rgb(218,218,235)'],[0.4, 'rgb(188,189,220)'],\
            [0.6, 'rgb(158,154,200)'],[0.8, 'rgb(117,107,177)'],[1.0, 'rgb(84,39,143)']]
scl2 = 'Viridis'

state_total_sales_2years['text'] = state_total_sales_2years['state'] + '<br>' +'total_sales_2years '+ state_total_sales_2years['state_name']

data = [ dict(
        type='choropleth',
        colorscale = scl,
        autocolorscale = False,
        locations = state_total_sales_2years['state'],
        z = state_total_sales_2years['total_sales_2years'].astype(float),
        locationmode = 'USA-states',
        text = state_total_sales_2years['text'],
        marker = dict(
            line = dict (
                color = 'rgb(255,255,255)',
                width = 2
            ) ),
        colorbar = dict(
            title = "total_sales_2years")
        ) ]

layout = dict(
        title = 'Total Sale State Wide',
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showlakes = True,
            lakecolor = 'rgb(255, 255, 255)'),
             )
    
fig = dict( data=data, layout=layout )
py.iplot( fig, filename='d3-cloropleth-map' )

High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~jonamjar/0 or inside your plot.ly account where it is named 'd3-cloropleth-map'


In [22]:
# Data for Dash

ntf_training_df['region'].value_counts()

North East         117067
South West         116962
Midwest (North)    100926
Mid-Atlantic        97222
South East          66310
Rockies             58693
Midwest (South)     25707
North West          12276
International        1495
Unknown               857
Name: region, dtype: int64

In [23]:
# prep data for dash interactive app

ntf_training_app_df = ntf_training_df[['first_order_date','total_sales_2years','region']]
ntf_training_app_df.head()

,first_order_date,total_sales_2years,region
0,2015-06-25 21:37:07,895.25,Rockies
1,2015-01-04 15:43:07,149.90,Midwest (South)
2,2015-01-11 13:19:22,77.97,South West
3,2015-01-11 22:19:22,24.00,North East
4,2015-01-14 10:19:22,280.68,Midwest (North)


In [24]:
def extract_month(row):
    return re.search('-([0-9][0-9])-',row).group(1)

In [25]:
# extracting month from first_order_date

ntf_training_app_df['month'] = ntf_training_app_df['first_order_date'].apply(extract_month)

/Users/mdobbali/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [26]:
# mapping months to its name

# month_dict = {
#         '01':'Jan','02':'Feb','03':'Mar',
#         '04':'Apr','05':'May','06':'Jun',
#         '07':'Jul','08':'Aug','09':'Sep',
#         '10':'Oct','11':'Nov','12':'Dec'
# }

# ntf_training_app_df['month'] = ntf_training_app_df['month'].map(month_dict)

In [27]:
#ntf_training_app_df.to_csv("resources/ntf_training_app_df.csv")

In [28]:
# ntf_training_app_df.groupby(['region','month'])['total_sales_2years'].mean().reset_index()

In [30]:
ntf_training_df.head()

,customer_id,zip_code,city,state,region,estimated_population,census_income,first_order_date,first_order_channel,first_order_md,first_order_brand,first_order_sales,num_orders_2years,total_sales_2years
0,1724258,84109,Salt Lake City,UT,Rockies,19819.0,48122.08,2015-06-25 21:37:07,CS,Men's,The North Face,0.00,8,895.25
1,105640784,72764,Springdale,AR,Midwest (South),33654.0,31827.30,2015-01-04 15:43:07,PLA,Ski,Giro,149.90,1,149.90
2,105669196,77301,Conroe,TX,South West,18057.0,25785.79,2015-01-11 13:19:22,PLA,Accessories,Sol Republic,77.97,1,77.97
3,105671761,02764,North Dighton,MA,North East,3638.0,45405.48,2015-01-11 22:19:22,Organic,Clothing Accessories,Oakley,24.00,1,24.00
4,105681094,57783,Spearfish,SD,Midwest (North),11707.0,31115.96,2015-01-14 10:19:22,Organic,Women's,The North Face,159.95,3,280.68
